In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import numpy as np
import galsim
import batsim
import time
import matplotlib.pyplot as plt

In [ ]:
psf = galsim.Gaussian(fwhm=1e-10, flux=1.0)

In [ ]:
psf.kValue(galsim.PositionD(2.0, 1.0))

In [ ]:
gal = galsim.Sersic(n=0.5, half_light_radius=1.5, flux=20).shear(g1=0.04, g2=-0.1)
# gal = galsim.Gaussian(half_light_radius=1.5, flux=20).shear(g1=0.04, g2=-0.1)
psf = galsim.Gaussian(fwhm=1.2, flux=1.0)
gal_conv = galsim.Convolve([gal, psf])
nn = 128
scale = 0.2
image = (
    gal.shift(0.5 * scale, 0.5 * scale)
    .drawImage(nx=nn, ny=nn, scale=scale, method="no_pixel")
    .array
)
image_psf = (
    psf.shift(0.5 * scale, 0.5 * scale)
    .drawImage(nx=nn, ny=nn, scale=scale, method="no_pixel")
    .array
)

image_conv = (
    gal_conv.shift(scale, scale)
    .drawImage(nx=nn / 4, ny=nn / 4, scale=scale * 2, method="no_pixel")
    .array
)

plt.close()
plt.imshow(image)
plt.colorbar()
print(scale)

In [ ]:
plt.close()
plt.imshow(image_conv)
plt.colorbar()

In [ ]:
image_conv2 = batsim._gsinterface.convolvePsf(
    scale, psf._sbp, image, 2, ngrid=int(nn / 4)
)
plt.close()
plt.imshow(image_conv2 - image_conv)
plt.colorbar()

In [ ]:
plt.imshow(image_conv2)
plt.colorbar()

In [ ]:
image_conv2.shape

In [ ]:
gal = (
    galsim.Sersic(n=1.0, half_light_radius=2.0, flux=20)
    .shear(g1=0.24, g2=-0.13)
    .shift(2 * scale, 3 * scale)
)
# gal = galsim.Gaussian(half_light_radius=1.5, flux=20).shear(g1=0.24, g2=-0.1)
psf = galsim.Gaussian(fwhm=1.3, flux=1.0)
gal_conv = galsim.Convolve([gal, psf])
nn = 1280
scale = 0.02
image = (
    gal.shift(0.5 * scale, 0.5 * scale)
    .drawImage(nx=nn, ny=nn, scale=scale, method="no_pixel")
    .array
)

image_psf = (
    psf.shift(0.5 * scale, 0.5 * scale)
    .drawImage(nx=nn, ny=nn, scale=scale, method="no_pixel")
    .array
)

scale2 = 0.2
nn2 = 64
image_conv = (
    gal_conv.shift(scale2 * 0.5, scale2 * 0.5)
    .drawImage(nx=nn2, ny=nn2, scale=scale2, method="no_pixel")
    .array
)

plt.close()
plt.imshow(image)
plt.colorbar()
print(scale)

In [ ]:
image_conv2 = batsim._gsinterface.convolvePsf(
    scale, psf._sbp, image, downsample_ratio=10, ngrid=nn2
)
image_conv3 = np.fft.ifft2(
    np.fft.fft2(image) * np.fft.fft2(np.fft.ifftshift(image_psf))
).real
plt.close()
plt.imshow(image_conv - image_conv2)
print(np.max(np.abs(image_conv - image_conv2)))
plt.colorbar()

In [ ]:
gal = galsim.Gaussian(half_light_radius=1.5, flux=20).shear(g1=0.04, g2=-0.1)
# gal = galsim.Sersic(n=0.5, half_light_radius=1.5, flux=20).shear(g1=0.04, g2=-0.1)
nn = 64
scale = 0.2
image = (
    gal.shift(0.5 * scale, 0.5 * scale)
    .drawImage(nx=nn, ny=nn, scale=scale, method="no_pixel")
    .array
)

x = np.fft.fftfreq(nn, scale / np.pi / 2.0)
y = np.fft.fftfreq(nn, scale / np.pi / 2.0)
inds = np.meshgrid(y, x, indexing="ij")
coords = np.vstack([np.ravel(_) for _ in inds[::-1]])
kygrid, kxgrid = np.meshgrid(y, x, indexing="ij")

image_fft = np.fft.fft2(np.fft.ifftshift(image))

ix = 32
for iy in range(nn):
    kx = kxgrid[iy, ix]
    ky = kygrid[iy, ix]
    print(image_fft[iy, ix] - gal.kValue(galsim.PositionD(kx, ky)))